In [1]:
import numpy as np
import pandas as pd
import torch
import ast

from tqdm.notebook import tqdm
from path import Path

In [2]:
#Stadium data for obtaining training data 
stadium_county_df = pd.read_csv('Cleaned_stadium_data.csv')

#Grouped data for running Bayesian SC
grouped_df = pd.read_csv('Grouped_df.csv')

county_covid = pd.read_csv('County_Covid_Data.csv')

county_covid['date'] = pd.to_datetime(county_covid['date'])
county_covid = county_covid.loc[~(county_covid['county'] == 'Unknown')]
county_covid = county_covid.loc[~(county_covid['cases'].isnull())]
county_covid['county'] = county_covid['county'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [77]:
home_state_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

reverse_home_state_dict = {v: k for k, v in home_state_dict.items()}
county_covid['state_abbrev'] = county_covid['state'].apply(lambda x: reverse_home_state_dict[x])

In [22]:
#Below function gets training X and training Y data for applying Bayesian Model 
def get_training_data(team_name_str, stadium_county_list, state_list, intervention_date_list, show_plot, week):
    assert type(stadium_county_list) == list
    assert type(state_list) == list
    assert type(intervention_date_list) == list

    # the arguments show_plot and week are never used

    # what does this do? remove whitespace?
    stadium_county_list = [x for x in stadium_county_list if x != ""]
    state_list = [x for x in state_list if x != ""]
    intervention_date_list = [x for x in intervention_date_list if x != ""]
    #Convert to lower case to avoid case insensitivity later
    stadium_county_list = [s.lower() for s in stadium_county_list]
    #To find intervention_date, we want first entry that has numbers, since some in data is text only.

    for s in intervention_date_list:
        if any(c.isdigit() for c in s):
            intervention_date = s

    intervention_date = pd.to_datetime(intervention_date)


    #Convert State Acronym to full state name
    for state in range(len(state_list)):
        if state_list[state] in home_state_dict:
            state_list[state] = home_state_dict[state_list[state]]
            
            
    #print(stadium_county_df)
    #Find Synthetic Counties
    synthetic_counties = list(stadium_county_df.loc[stadium_county_df['Team'] == team_name_str]['Donor_Counties'])[0]#.copy()
    #synthetic_counties = list(synthetic_counties)
    synthetic_counties = eval(synthetic_counties)
    synthetic_counties = [s.lower() for s in synthetic_counties]
    n_donors = len(synthetic_counties)
    
    #print(synthetic_counties)

    #Find Dataframe of X and Y data
    #Special case where 
    
    
    if team_name_str == 'Washington':
        stadium_county_data = county_covid.loc[(county_covid['county'].isin(stadium_county_list)) | ((county_covid['county'].isin(synthetic_counties)) & (county_covid['state'] == 'Maryland'))]
    else:
        stadium_county_data = county_covid.loc[(county_covid['county'].isin(stadium_county_list) | (county_covid['county'].isin(synthetic_counties))) & (county_covid['state'].isin(state_list))]
    
    stadium_county_data = stadium_county_data.fillna(method='bfill')
    stadium_county_data['date'] = pd.to_datetime(stadium_county_data['date'], infer_datetime_format=True)
    
    earliest_date = list(stadium_county_data.loc[stadium_county_data['county'].isin(stadium_county_list)]['date'])[0]
    
    #Start training from the earliest date of when our stadium county data becomes available.
    stadium_county_data = stadium_county_data.loc[stadium_county_data['date'] >= earliest_date]
    
    
    #Total Pivot is pivot table cases for entire dataset, training pivot is the same but for < intervention date
    total_pivot = stadium_county_data.pivot_table(columns='county', values='cases', index= 'date').reset_index()
    total_pivot = total_pivot.loc[total_pivot['date'] >= earliest_date]
    
    #Sum up stadium counties for our prediction. 
    total_pivot['Stadium_County'] = total_pivot.apply(lambda row: row[stadium_county_list].sum(), axis=1)
    #total_pivot['Stadium_County'] = total_pivot.loc[:, total_pivot.columns == (stadium_county_list[0])]
    
    
    total_pivot.drop(stadium_county_list, axis=1, inplace=True)
    
    total_pivot.fillna(0, inplace=True)
    
    training_pivot = total_pivot.loc[total_pivot['date'] < intervention_date]
    
    training_dates = training_pivot['date']
    
    total_dates = total_pivot['date']
    
    test_pivot = total_pivot.loc[total_pivot['date'] >= intervention_date]
    
    test_pivot = test_pivot.drop(['date'], axis=1)
    
    training_pivot = training_pivot.drop(['date'], axis=1)
    
    total_pivot = total_pivot.drop(['date'], axis=1)
    
    
    X_train = training_pivot.loc[:, ~training_pivot.columns.isin(['Stadium_County'])]
    
    Y_train = training_pivot['Stadium_County']
    
    total_X = total_pivot.loc[:, ~total_pivot.columns.isin(['Stadium_County'])]
    total_Y = total_pivot['Stadium_County']
    
    test_X = test_pivot.loc[:, ~test_pivot.columns.isin(['Stadium_County'])]
    test_Y = test_pivot['Stadium_County']
    

    return X_train, Y_train, test_X, test_Y, total_dates

x_train, y_train, x_test, y_test, dates = get_training_data('Cincinnati', ["Hamilton"], ["OH"], ['10/04/2020'], True, 0)

['hamilton'] ['OH'] 2020-10-04 00:00:00


In [5]:
p = Path('dat')
p.mkdir_p()

Path('dat')

In [39]:
from copy import deepcopy
zipped_input = zip(grouped_df['Team'], grouped_df['County(s)'], grouped_df['State'], grouped_df['First date home stadium open to fans'])

for i, (team, county, state, date) in tqdm(enumerate(zipped_input)):
    if team in ['Arizona']:
        continue
    
    outpath = Path('dat').joinpath(team)
    outpath.mkdir_p()

    date = ast.literal_eval(date)
    county = ast.literal_eval(county)
    state = ast.literal_eval(state)

    # x are donors, y are stadiums, train is pre-intervention, test is post intervention
    x_train, y_train, x_test, y_test, dates = get_training_data(team, county, state, date, True, 0)
    x_train, y_train, x_test, y_test = torch.tensor(x_train.values),torch.tensor(y_train.values), torch.tensor(x_test.values), torch.tensor(y_test.values)

    # Combine Donor and Stadium in our traning data
    train_data = torch.cat((x_train, torch.unsqueeze(y_train, 1)), dim=1)  # this is a T x N matrix
    test_data = torch.cat((x_test, torch.unsqueeze(y_test, 1)), dim=1)  # this is a T x N matrix

    # Save data
    torch.save(train_data, outpath.joinpath('train_data.pt'))
    torch.save(test_data, outpath.joinpath('test_data.pt'))

'09-10-2023'

### Clean up `Grouped_df.csv`

In [62]:
grouped_df = pd.read_csv('Grouped_df.csv')
del grouped_df['Unnamed: 0']

def get_clean_list_from_string(x):
    """Removes all empty elements of list attributes"""
    return list(filter(lambda x: x != '', ast.literal_eval(x)))

for col in ['County(s)', 'State', 'First date home stadium open to fans']:
    grouped_df[col] = grouped_df[col].apply(get_clean_list_attr)

def get_clean_date(x):
    date_str_list = ast.literal_eval(x)
    clean_date_str_list = []
    for date_str in date_str_list:
        try:
            ts = pd.Timestamp(date_str)
        except ValueError:
            ts = pd.NaT
        if ts is not pd.NaT:
            clean_date_str_list.append(ts.strftime('%m-%d-%Y'))
    assert len(set(clean_date_str_list)) == 1
    return clean_date_str_list[0]

grouped_df['intervention_date'] = grouped_df['First date home stadium open to fans'].apply(get_clean_date)

grouped_df

,Team,First date home stadium open to fans,County(s),State,intervention_date
0,Arizona,['10/25/2020'],['maricopa'],['AZ'],10-25-2020
1,Atlanta,['10/25/2020'],['fulton'],['GA'],10-25-2020
2,Baltimore,['11/1/2020'],"['baltimore', 'baltimore city']","['MD', 'MD']",11-01-2020
3,Buffalo,"['Post season fans', 'First home', 'game no fa...",['erie'],['NY'],09-13-2020
4,Carolina,['10/4/2020'],['mecklenburg'],['NC'],10-04-2020
5,Chicago,['9/20/2020'],['cook'],['IL'],09-20-2020
6,Cincinnati,['10/4/2020'],['hamilton'],['OH'],10-04-2020
7,Cleveland,['9/17/2020'],['cuyahoga'],['OH'],09-17-2020
8,Dallas,['9/20/2020'],"['dallas', 'tarrant']","['TX', 'TX']",09-20-2020
9,Denver,['9/27/2020'],"['denver', 'arapahoe']","['CO', 'CO']",09-27-2020


### Create a new dataframe where each row is a stadium county (not a team)

In [185]:
expanded_rows = []
for i, row in tqdm(grouped_df.iterrows(), total=len(grouped_df)):
    team = row['Team']
    date = row['intervention_date']
    counties = ast.literal_eval(row['County(s)'])
    states = ast.literal_eval(row['State'])
    for county, state in zip(counties, states):
        if state == 'District of Columbia':
            state = 'DC'
        expanded_rows.append((team, date, county, state))
df_stadium_counties = pd.DataFrame(expanded_rows, columns=['team', 'intervention_date', 'county', 'state'])
df_stadium_counties

  0%|          | 0/32 [00:00<?, ?it/s]

,team,intervention_date,county,state
0,Arizona,10-25-2020,maricopa,AZ
1,Atlanta,10-25-2020,fulton,GA
2,Baltimore,11-01-2020,baltimore,MD
3,Baltimore,11-01-2020,baltimore city,MD
4,Buffalo,09-13-2020,erie,NY
5,Carolina,10-04-2020,mecklenburg,NC
6,Chicago,09-20-2020,cook,IL
7,Cincinnati,10-04-2020,hamilton,OH
8,Cleveland,09-17-2020,cuyahoga,OH
9,Dallas,09-20-2020,dallas,TX


### Save train and test pivot tables

In [193]:
for team_name_str in tqdm(df_stadium_counties['team'].unique()):
    if team_name_str == 'Arizona':
        continue

    intervention_date = pd.Timestamp(df_stadium_counties[df_stadium_counties['team'] == team_name_str]['intervention_date'].values[0])

    stadium_county_states = df_stadium_counties[df_stadium_counties['team'] == team_name_str]['state'].to_list()
    if team_name_str == 'Washington':
        stadium_county_states.append('MD')

    stadium_county_names = df_stadium_counties[df_stadium_counties['team'] == team_name_str]['county'].to_list()
    stadium_county_names = [s.lower() for s in stadium_county_names]

    donor_county_names = ast.literal_eval(stadium_county_df[stadium_county_df['Team']==team_name_str].iloc[0]['Donor_Counties'])
    donor_county_names = [s.lower() for s in donor_county_names]

    # stadium_covid_data = county_covid.loc[county_covid['county'].isin(stadium_county_names) & county_covid['state_abbrev'].isin(stadium_county_states)]
    # donor_covid_data = county_covid.loc[county_covid['county'].isin(donor_county_names) & county_covid['state_abbrev'].isin(stadium_county_states)]

    # this is bad logic... we need to find the state abbrev for donors
    # old name; this combines the stadium and donor counties
    stadium_county_data = county_covid.loc[county_covid['county'].isin(donor_county_names + stadium_county_names) & county_covid['state_abbrev'].isin(stadium_county_states)].copy()
    # stadium_county_data = stadium_county_data.fillna(method='bfill')
    stadium_county_data['date'] = pd.to_datetime(stadium_county_data['date'], infer_datetime_format=True)

    earliest_date = stadium_county_data.loc[stadium_county_data['county'].isin(stadium_county_names)]['date'].min()

    #Start training from the earliest date of when our stadium county data becomes available.
    stadium_county_data = stadium_county_data.loc[stadium_county_data['date'] >= earliest_date]

    #Total Pivot is pivot table cases for entire dataset, training pivot is the same but for < intervention date
    total_pivot = stadium_county_data.pivot_table(columns='county', values='cases', index= 'date')
    # take only the rows greater than earliest_date
    total_pivot = total_pivot.loc[total_pivot.index >= earliest_date]

    # #Sum up stadium counties for our prediction. 
    total_pivot['Stadium_County'] = total_pivot.apply(lambda row: row[stadium_county_names].sum(), axis=1)
    total_pivot.drop(stadium_county_names, axis=1, inplace=True)
    total_pivot.fillna(0, inplace=True)
    train_pivot = total_pivot.loc[total_pivot.index < intervention_date]
    test_pivot = total_pivot.loc[total_pivot.index >= intervention_date]

    train_pivot.to_csv(f'dat/{team_name_str}/train_pivot.csv', index=True)
    test_pivot.to_csv(f'dat/{team_name_str}/test_pivot.csv', index=True)


  0%|          | 0/32 [00:00<?, ?it/s]

### Test that the new pivot tables are the same as the old arrays

In [203]:
for team_name_str in tqdm(df_stadium_counties['team'].unique()):
    if team_name_str == 'Arizona':
        continue
    data_dir = Path(f'dat/{team_name_str}')
    # load pivot table where first column is index
    train_pivot = pd.read_csv(data_dir.joinpath('train_pivot.csv'), index_col=0)
    train_data = np.array(torch.load(data_dir.joinpath('train_data.pt')))
    assert np.allclose(train_data, train_pivot.values)

    test_pivot = pd.read_csv(data_dir.joinpath('test_pivot.csv'), index_col=0)
    test_data = np.array(torch.load(data_dir.joinpath('test_data.pt')))
    assert np.allclose(test_data, test_pivot.values)

  0%|          | 0/32 [00:00<?, ?it/s]